In [ ]:
!pip install langchain -q
!pip install pypdf -q
!pip install sentence-transformers -q
!pip install chromadb -q
!pip install openai -q
!pip install langchain_community -q
!pip install llamaapi -q
!pip install langchain_experimental -q

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.embeddings import HuggingFaceEmbeddings
from llamaapi import LlamaAPI
from langchain_experimental.llms import ChatLlamaAPI
import warnings
warnings.filterwarnings("ignore")

In [ ]:
loader = PyPDFLoader("data.pdf")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=128)
texts = text_splitter.split_documents(data)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory="chroma_store"
)

In [ ]:
llama = LlamaAPI("") # Input your api key
llm = ChatLlamaAPI(client=llama)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)
while True:
    query = input()
    if query == "exit":
        break
    result = qa({"query": query})
    print("LLAMA:", result["result"])